In [1]:
#r "nuget: Azure.Search.Documents, 11.7.0-beta.7"
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget:dotenv.net, 4.0.0"

Installed Packages Azure.Identity, 1.15.0 Azure.Search.Documents, 11.7.0-beta.7 dotenv.net, 4.0.0

# Define env constants

In [103]:
using Azure.Identity;
using Azure.AI.OpenAI;
using Azure.AI.OpenAI.Chat;
using OpenAI.Chat;

string aoaiEmbeddingModel = "text-embedding-3-large";
string aoaiEmbeddingDeployment = "text-embedding-3-large";
string aoaiGptModel = "gpt-5-mini";
string aoaiGptDeployment = "gpt-5-mini";

string indexName = "productx-report-index";
string knowledgeSourceName = $"{indexName}-source";
string knowledgeAgentName = $"{indexName}-agent";

var credential = new DefaultAzureCredential(includeInteractiveCredentials: true);

In [81]:
string aoaiEndpoint = "https://testmediumazureopenai.openai.azure.com/";
string searchEndpoint = "https://ai-search-medium.search.windows.net";

## Define IndexKnowledge Source

In [90]:
using Azure.Search.Documents.Indexes.Models;

// Create the knowledge source
var indexKnowledgeSource = new SearchIndexKnowledgeSource(
    name: knowledgeSourceName,
    searchIndexParameters: new SearchIndexKnowledgeSourceParameters(searchIndexName: indexName)
    {
        SourceDataSelect = "chunk_id,chunk,title"
    }
);

await indexClient.CreateOrUpdateKnowledgeSourceAsync(indexKnowledgeSource);

Console.WriteLine($"Knowledge source '{knowledgeSourceName}' created or updated successfully.");

Knowledge source 'productx-report-index-source' created or updated successfully.


# Define Knowledge Agent

In [96]:
using Azure.Search.Documents.Indexes.Models;
using Azure.Search.Documents.Models;

var openAiParameters = new AzureOpenAIVectorizerParameters
{
    ResourceUri = new Uri(aoaiEndpoint),
    DeploymentName = aoaiGptDeployment,
    ModelName = aoaiGptModel
};

var agentModel = new KnowledgeAgentAzureOpenAIModel(azureOpenAIParameters: openAiParameters);

var outputConfig = new KnowledgeAgentOutputConfiguration
{
    Modality = KnowledgeAgentOutputConfigurationModality.AnswerSynthesis,
    IncludeActivity = true
};

// Create the knowledge agent
var agent = new KnowledgeAgent(
    name: knowledgeAgentName,
    models: new[] { agentModel },
    knowledgeSources: new KnowledgeSourceReference[] { 
        new KnowledgeSourceReference(knowledgeSourceName) 
        {
             IncludeReferences = true, IncludeReferenceSourceData = true, RerankerThreshold = (float?)2.5 }
        }
)
{
    OutputConfiguration = outputConfig
};

await indexClient.CreateOrUpdateKnowledgeAgentAsync(agent);
Console.WriteLine($"Knowledge agent '{knowledgeAgentName}' created or updated successfully.");

Knowledge agent 'productx-report-index-agent' created or updated successfully.


# Test Agent

In [100]:
string instructions = @"
You are a Q&A agent that can answer questions about the Product report. Your knowledge is strictly limited to the information contained within the provided text.

Carefully analyze the user's question and the context from the report.

If the answer is present in the provided text, give a concise and accurate response based only on that information.

If the answer cannot be found in the provided text, you must respond with the exact phrase: 'I don't know'.

Do not make assumptions or use any external knowledge.
";

var messages = new List<Dictionary<string, string>>
{
    new Dictionary<string, string>
    {
        { "role", "system" },
        { "content", instructions }
    }
};

In [101]:
using Azure.Search.Documents.Agents;
using Azure.Search.Documents.Agents.Models;

var agentClient = new KnowledgeAgentRetrievalClient(
    endpoint: new Uri(searchEndpoint),
    agentName: knowledgeAgentName,
    tokenCredential: new DefaultAzureCredential()
);

messages.Add(new Dictionary<string, string>
{
    { "role", "user" },
    { "content", @"
Write marketingemail for Product X based on the customer insights?
" }
});

var retrievalResult = await agentClient.RetrieveAsync(
    retrievalRequest: new KnowledgeAgentRetrievalRequest(
        messages: messages
            .Where(message => message["role"] != "system")
            .Select(
            message => new KnowledgeAgentMessage(content: new[] { new KnowledgeAgentMessageTextContent(message["content"]) })  { Role = message["role"] }
            )
            .ToList()
    )
);

messages.Add(new Dictionary<string, string>
{
    { "role", "assistant" },
    { "content", (retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text }
});

In [102]:
(retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text 

Subject: Why customers love Product X — comfort, long battery, and smarter coaching

Hi there,

Information was found showing that 82% of customers rated Product X four stars or higher — including 58% who gave it five stars — highlighting broad satisfaction among users [ref_id:0][ref_id:4]. Comfort, battery life, and seamless app integration were the most praised attributes, and comfort showed the strongest link to repeat purchases and positive referrals [ref_id:0][ref_id:4].

Choose the Product X that fits your life:
- Product X Active — core tracking (steps, heart rate, calories) with 10-day battery life, great for everyday users and fitness beginners, priced at $99 [ref_id:0].
- Product X Pro — all Active features plus SpO2, HRV, advanced sleep analytics, VO2 Max, and adaptive coaching; the Pro model accounted for 40% of new unit sales and has a 7-day battery life, priced at $179 [ref_id:0].
- Product X Kids — step tracking, active minutes, sleep monitoring, gamified goals, and a pa